<a href="https://colab.research.google.com/github/janaahmeed/codealpha_tasks/blob/main/codeAlpha_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import sklearn


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
Path to dataset files: /kaggle/input/ravdess-emotional-speech-audio


After framing, each frame is multiplied by a Hamming window to reduce spectral leakage before silence detection and further feature extraction.  
# When you use librosa.feature.mfcc, framing + windowing (Hamming) are already applied internally by default.

y>> the audio time-series signal , Compute MFCC features from audio signal lf sampled at fs Hz
y(t) = audio_signal
mfcc(y=audio_signal)

In [7]:

import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

x, y = [], []
plt.rcParams['figure.figsize'] = (18,4)

for root, dirs, files in os.walk(path):
    for _file in files:
        if _file.endswith(".wav"):
            file_path = os.path.join(root, _file)

            lf, fs = librosa.load(file_path, duration=0.03)

            y_trimmed, _ = librosa.effects.trim(lf ,top_db=25 )  # 20–30 is common for speech

            mfccs = librosa.feature.mfcc(y=y_trimmed, sr=fs, window='hamming')
            mfccs = mfccs.T # (time_steps, n_mfcc)
            # Why? LSTM expects time dimension first

            x.append(mfccs)
            y.append(_file.split("-")[2])

#(n_mfcc, time_frames) >>shape mfcc


#NumPy arrays require equal shapes
#Your MFCCs are variable-length sequences >> so we neednot ( Pad for CNN / Use sequence models (LSTM) / Use tf.keras.preprocessing.sequence.pad_sequences)
#“Don’t force uniform shape — store each MFCC as a separate object” if duration is fixed we donot need for it
#so type object
x = np.array(x, dtype=object)
y = np.array(y)

print(x.shape, y.shape)




/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=662
  warnings.warn(


(2880, 2, 20) (2880,)


In [6]:
import tensorflow as tf
x=tf.keras.preprocessing.sequence.pad_sequences(x ,padding='post')

#for conv2d new dimansion to adapt MFCC inputs for convolutional layers.
x=x[..., np.newaxis]


//CNN / LSTM